In [130]:
import pandas as pd

# read source file snapshots
df = pd.read_excel(r'G:\DM_AutoWrite\l_compare_snapshots_20230224_Test.xlsx', sheet_name = 'differences')

# read als file
df1 = pd.read_excel(r'G:\DM_AutoWrite\BGB-A317-306_V9.0_CC7_9Dec20_1053.xlsx',sheet_name='Fields')


In [131]:
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 30)

In [132]:
df

,Project,Site Number,Subject,Instance Name,Data Set Name,CRF Page Name,Record Id,Record Position,Lab Form,Analyte Name,Variable Name,Variable Label,Previous Value,Current Value,Status,Record Date,Cut off Date,Newflag,Comments,Closed?
0,BGB-A317-306,34026,034026-003,Unscheduled Visits 17 Sep 2020,ADASAMP,Anti-tislelizumab antibodies,129931798,0,NaN,NaN,ADADAT_RAW,Sample Collection Date (Character),17 SEP 2021,17 SEP 2020,Updated,17 SEP 2020,28Feb2022,Old,YY_7/11/2022: Record in date change tracker,Open
1,BGB-A317-306,7025,007025-002,Adverse Events,AE,Adverse Events,52205145,3,NaN,NaN,AEOUT,Outcome,Recovering/Resolving,Unknown,Updated,12 DEC 2019,28Feb2022,Old,YY_8/4/2022: Record in Tracker,Open
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10422,BGB-A317-306,86113,086113-013,Cycle 11 Day 1 (1),VS3,Vital Signs<Log/>,76211466,1,NaN,NaN,VS3DAT_RAW,Date and time of Assessment (Character),6 AUG 2020 11:35,6 AUG 2020 11:28,Updated,6 AUG 2020,28Feb2022,Old,YY_1/3/2023: Record in Tracker,open
10423,BGB-A317-306,86133,086133-019,Cycle 9 Day 1 (1),VS3,Vital Signs<Log/>,56636922,1,NaN,NaN,VSWTRRES_RAW,Weight (Character),52,48,Updated,19 FEB 2020,28Feb2022,Old,YY_11/4/2022: Record in Tracker,Open


In [133]:
df1

,FormOID,FieldOID,Ordinal,DraftFieldNumber,DraftFieldName,DraftFieldActive,VariableOID,DataFormat,DataDictionaryName,UnitDictionaryName,CodingDictionary,ControlType,AcceptableFileExtensions,IndentLevel,PreText,...,DoesNotBreakSignature,LowerRange,UpperRange,NCLowerRange,NCUpperRange,ViewRestrictions,EntryRestrictions,ReviewGroups,IsVisualVerify,FDownloadedFromObjectId,FSourceObjectId,VDownloadedFromObjectId,VSourceObjectId,FSourceUrlId,VSourceUrlId
0,SUBJECT,SUBJID,1,NaN,SUBJID,True,SUBJID,$15,NaN,NaN,NaN,Text,NaN,0,Subject number,...,False,NaN,NaN,NaN,NaN,NaN,NaN,Data Management,False,7466485,6963714,6699783.0,6247376.0,NaN,NaN
1,SUBJECT,COUNTRY,6,NaN,COUNTRY,True,COUNTRY,$20,NaN,NaN,NaN,Text,NaN,0,Country,...,True,NaN,NaN,NaN,NaN,NaN,"Coder Import Role,OL Role,Clinical Research As...",NaN,False,7466484,6963713,6699784.0,6247377.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1078,ICBO,ICBODAT,2,NaN,ICBODAT,True,ICBODAT,dd MMM yyyy,NaN,NaN,NaN,DateTime,NaN,0,Date blood biomarkers consent signed,...,False,NaN,NaN,NaN,NaN,NaN,NaN,Data Management,False,7465795,6963024,6700697.0,6248290.0,NaN,NaN
1079,IH,SURDTC1,1,NaN,SURDTC1,True,SURDTC1,dd MMM yyyy,NaN,NaN,NaN,DateTime,NaN,0,Survival Follow-up Visit,...,False,NaN,NaN,NaN,NaN,NaN,NaN,Data Management,False,7465821,6963050,6700652.0,6248245.0,NaN,NaN


#### Preparation

In [134]:
#### df1 -> df2(FormOID, VariableOID, Ordinal, IsLog)
df2 = pd.DataFrame(df1,columns=['FormOID','DraftFieldName','Ordinal','IsLog'])
df2

,FormOID,DraftFieldName,Ordinal,IsLog
0,SUBJECT,SUBJID,1,False
1,SUBJECT,COUNTRY,6,False
...,...,...,...,...
1078,ICBO,ICBODAT,2,False
1079,IH,SURDTC1,1,False


In [135]:
#### create a ref col'VarOID_Ref', with removing suffixes
def RemoveSuffix(x):
    suffix_list = ['_RAW','_LLT','_HLGT','_HLT','_PT','_SOC','_ATC4','_ATC3','_ATC2','_ATC','_PRODSYN','_PROD']
    action_taken = False
    for item in suffix_list:
        if x.__contains__(item):
            return x.replace(item,' ').strip()
            action_taken = True
    if action_taken == False:
        return x
    
df['VarOID_Ref'] = df.apply(lambda x: RemoveSuffix(x['Variable Name']),axis=1)

df

,Project,Site Number,Subject,Instance Name,Data Set Name,CRF Page Name,Record Id,Record Position,Lab Form,Analyte Name,Variable Name,Variable Label,Previous Value,Current Value,Status,Record Date,Cut off Date,Newflag,Comments,Closed?,VarOID_Ref
0,BGB-A317-306,34026,034026-003,Unscheduled Visits 17 Sep 2020,ADASAMP,Anti-tislelizumab antibodies,129931798,0,NaN,NaN,ADADAT_RAW,Sample Collection Date (Character),17 SEP 2021,17 SEP 2020,Updated,17 SEP 2020,28Feb2022,Old,YY_7/11/2022: Record in date change tracker,Open,ADADAT
1,BGB-A317-306,7025,007025-002,Adverse Events,AE,Adverse Events,52205145,3,NaN,NaN,AEOUT,Outcome,Recovering/Resolving,Unknown,Updated,12 DEC 2019,28Feb2022,Old,YY_8/4/2022: Record in Tracker,Open,AEOUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10422,BGB-A317-306,86113,086113-013,Cycle 11 Day 1 (1),VS3,Vital Signs<Log/>,76211466,1,NaN,NaN,VS3DAT_RAW,Date and time of Assessment (Character),6 AUG 2020 11:35,6 AUG 2020 11:28,Updated,6 AUG 2020,28Feb2022,Old,YY_1/3/2023: Record in Tracker,open,VS3DAT
10423,BGB-A317-306,86133,086133-019,Cycle 9 Day 1 (1),VS3,Vital Signs<Log/>,56636922,1,NaN,NaN,VSWTRRES_RAW,Weight (Character),52,48,Updated,19 FEB 2020,28Feb2022,Old,YY_11/4/2022: Record in Tracker,Open,VSWTRRES


In [136]:
#### df2 +-> df
#df = pd.merge(df, df2, how = 'left', left_on=['Data Set Name','Variable Name'], right_on=['FormOID','VariableOID'])
df = pd.merge(df, df2, how = 'left', left_on=['VarOID_Ref'], right_on=['DraftFieldName'])
df

,Project,Site Number,Subject,Instance Name,Data Set Name,CRF Page Name,Record Id,Record Position,Lab Form,Analyte Name,Variable Name,Variable Label,Previous Value,Current Value,Status,Record Date,Cut off Date,Newflag,Comments,Closed?,VarOID_Ref,FormOID,DraftFieldName,Ordinal,IsLog
0,BGB-A317-306,34026,034026-003,Unscheduled Visits 17 Sep 2020,ADASAMP,Anti-tislelizumab antibodies,129931798,0,NaN,NaN,ADADAT_RAW,Sample Collection Date (Character),17 SEP 2021,17 SEP 2020,Updated,17 SEP 2020,28Feb2022,Old,YY_7/11/2022: Record in date change tracker,Open,ADADAT,ADASAMP,ADADAT,2.0,False
1,BGB-A317-306,7025,007025-002,Adverse Events,AE,Adverse Events,52205145,3,NaN,NaN,AEOUT,Outcome,Recovering/Resolving,Unknown,Updated,12 DEC 2019,28Feb2022,Old,YY_8/4/2022: Record in Tracker,Open,AEOUT,AE,AEOUT,9.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10878,BGB-A317-306,86133,086133-019,Cycle 9 Day 1 (1),VS3,Vital Signs<Log/>,56636922,1,NaN,NaN,VSWTRRES_RAW,Weight (Character),52,48,Updated,19 FEB 2020,28Feb2022,Old,YY_11/4/2022: Record in Tracker,Open,VSWTRRES,VS1,VSWTRRES,6.0,False
10879,BGB-A317-306,86133,086133-019,Cycle 9 Day 1 (1),VS3,Vital Signs<Log/>,56636922,1,NaN,NaN,VSWTRRES_RAW,Weight (Character),52,48,Updated,19 FEB 2020,28Feb2022,Old,YY_11/4/2022: Record in Tracker,Open,VSWTRRES,VS3,VSWTRRES,2.0,False


In [137]:
#### fill empty value
df.fillna('99x083x', inplace = True)

In [138]:
#### outout
df.to_excel(r'G:\DM_AutoWrite\df.xlsx',index=False)